# Bolivian Football League Betting Odds
## 4. Bet Simulator

This notebook utilizes the odds predicted by the betting model to come up with a profitable betting strategy based on the Kelly Criterion

- https://insights.matchbook.com/the-kelly-criterion/

In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
import string
import matplotlib.ticker as ticker
import re
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# load dataset
odds = pd.read_csv('/Users/miketondu/Dropbox/Data Science/Sharpest Minds/odds_data/bolivian_football_odds_prepared.csv',
                  parse_dates= ['date'])

# drop unwanted columns

odds.drop([ 'cumulative_home_win', 'cumulative_away_win',
            'cumulative_home_loss', 'cumulative_away_loss','cumulative_home_points',
            'cumulative_away_points','cumulative_home_games_played',
            'cumulative_away_games_played','cumulative_home_goals_allowed', 'cumulative_away_goals_allowed',
            'cumulative_home_win_first_h',
            'cumulative_away_win_first_h', 'cumulative_home_win_second_h',
            'cumulative_away_win_second_h', 'cumulative_home_goals',
            'cumulative_away_goals',]
          ,axis=1,inplace=True)

# unique matches
odds_unique = odds.drop_duplicates('match_href')

In [2]:
odds_unique.columns

Index(['match', 'match_href', 'bookmakers', 'home_odds', 'draw_odds',
       'away_odds', 'payout', 'date', 'time', 'season_year', 'home_team',
       'away_team', 'implied_home_odds', 'implied_draw_odds',
       'implied_away_odds', 'final_result', 'home_goals', 'away_goals',
       'first_half_home', 'first_half_away', 'second_half_home',
       'second_half_away', 'home_goals_allowed', 'away_goals_allowed',
       'total_goals', 'over', 'total_first_half', 'total_second_half',
       'win_home_or_away', 'winner', 'home_win', 'away_win', 'home_loss',
       'away_loss', 'home_win_first_h', 'away_win_first_h',
       'home_win_second_h', 'away_win_second_h', 'home_points', 'away_points',
       'home_games_played', 'away_games_played', 'upset', 'altitude',
       'tropics', 'season_win_home_team', 'season_win_away_team',
       'season_loss_home_team', 'season_loss_away_team',
       'season_points_home_team', 'season_points_away_team',
       'season_games_played_home_team', 'season_

In [5]:
def upset_wager(df, starting_bankroll, wagering_size):
    upsets = 0
    non_upsets = 0
    bankroll = starting_bankroll
    for game in df:
        if df.home_odds[game] > df.away_odds[game]:
            if df.home_goals[game] > df.away_goals[game]:
                upsets +=1
                bankroll += wagering_size * (df.home_odds -1)
    
    

In [15]:
odds.home_odds[6]

1.7

#### What're we trying to achieve:
- Function that compares odds predicted by model against the casino odds and uses Kelly Criterion formula in order to set a betting stake
- This stake is then used in the gambling simulator function in order to simulate a season of betting

In [33]:
def kelly(odds,p):
    ''' Kelly criterion formula outputs the fraction of your bankroll to bet depending on the odds predicted
    f = fraction of the bankroll to bet
    b = decimal odds -1
    p = implied probability of our model
    q = probability of losing (1-p)
     '''
    b = odds - 1
    f = (b*p-(1-p))/b
    
    return print('Percentage of bankroll to bet:', round(f*100,2), '%')
    
kelly(1.80,.6)
    
    

Percentage of bankroll to bet: 10.0 %


In [36]:
def bet_simulator(dataframe, starting_bankroll, wagering_size):
    upsets = 0
    non_upsets = 0

    bankroll = starting_bankroll

    for game in dataframe:
        home_team = game['home_team']
        away_team = game['away_team']

        home_goals = game['home_goals']
        away_goals = game['away_goals']

        home_odds = game['home_odds']
        draw_odds = game['draw_odds']
        away_odds = game['away_odds']

        if home_odds > away_odds:
            if home_goals > away_goals:
                upsets += 1
                bankroll += wagering_size * (home_odds - 1)
            else:
                non_upsets += 1
                bankroll -= wagering_size

    ROI = ((bankroll - starting_bankroll) / (wagering_size * (upsets + non_upsets))) * 100

    print ("There were '%s' upsets out of '%s' total matches" % (upsets, upsets + non_upsets))
    print ("Starting bankroll = '%s'" % (starting_bankroll))
    print ("Finishing bankroll = '%s' | ROI = '%s'" % (bankroll, ROI))

In [37]:
bet_simulator(odds_unique, 1000, 5)

TypeError: string indices must be integers